# Is the following _k_-MTSLIA a correct one?

Well, is it?

In [115]:
import random
import itertools
import collections

In [116]:
def extract_alphabet(data):
    alpha = set()
    for datum in data:
        alpha |= set(datum)
    return alpha

def extract_local_k_grams(data, k, edges):
    attested = set()
    for datum in data:
        datum = (edges[0],) + tuple(datum) + (edges[1],) 
        for offset in range(len(datum) - k + 1):
            attested.add(tuple(datum[offset:(offset + k)]))
    return attested

def valid_k_gram(k_gram, edges):
    k = len(k_gram)
    return any(True for L, R in itertools.combinations(range(k + 1), 2) \
               if  all(k_gram[i] == edges[0]  for i in range(L)) \
               and all(k_gram[i] == edges[1]  for i in range(R, k)) \
               and not any(k_gram[i] in edges for i in range(L, R)))

def mtslia(data, k=2, edges=('>', '<')):
    alpha = extract_alphabet(data)
    attested = extract_local_k_grams(data, k, edges)
    grammar = collections.defaultdict(set)
    for k_gram in itertools.product(alpha | {*edges}, repeat=k):
        if k_gram in attested:
            continue
        if not valid_k_gram(k_gram, edges):
            continue
        projection = []
        for m, datum in enumerate(data):
            projection += [(m, -(i + 1), edges[0]) for i in range(k)]
            for n, segment in enumerate(datum):
                if segment in k_gram:
                    projection += [(m, n, segment)]
            projection += [(m, len(datum) + i, edges[1]) for i in range(k)]
        projection.sort()
        blocker_sets = set()
        for offset in range(len(projection) - k + 1):
            M, N, segments = zip(*projection[offset:(offset + k)])
            if segments == k_gram and all(m == M[0] for m in M):
                blocker_set = set()
                for n1, n2 in zip(N[:-1], N[1:]):
                    blocker_set |= set(data[M[0]][(n1 + 1):n2])
                blocker_sets.add(tuple(sorted(blocker_set)))
        blockers = set(blocker for blocker_set in blocker_sets for blocker in blocker_set)
        if blocker_sets:
            if () in blocker_sets:
                continue
            for blocker in sorted(blockers, reverse=True):
                if all(blockers - {blocker} & set(blocker_set) for blocker_set in blocker_sets):
                    blockers -= {blocker}
        tier = tuple(sorted({*k_gram, *blockers} - {*edges}))
        grammar[tier].add(k_gram)
    return dict(grammar)

# A 100% fictitious dataset

```
i       u    High (Height 3)
  e   o      Mid  (Height 2)
    a        Low  (Height 1)
```

 - Constraint 1: any stretch of 3 syllables cannot have all 3 heights
 - Constraint 2: front-back harmony; /a/ is neutral

In [117]:
data = ['dununono', 'dudo', 'toto', 'notato', 'deninen', 'nenenat', 'idde', 'inintenedi']

## 2-MTSLIA's take on the dataset

In [118]:
mtslia(data, 2)

{('e', 'i', 'n', 'o', 'u'): {('n', 'n')},
 ('d', 'e', 'n'): {('>', 'e'), ('n', 'd')},
 ('a', 'n'): {('>', 'a'), ('a', 'n')},
 ('a',): {('a', 'a')},
 ('a', 'd'): {('a', 'd'), ('d', 'a')},
 ('a', 'u'): {('a', 'u'), ('u', 'a')},
 ('a', 'e'): {('a', 'e')},
 ('a', 'i'): {('a', 'i'), ('i', 'a')},
 ('a', 'o', 't'): {('a', 'o'), ('o', 'a'), ('t', 't')},
 ('a', 't'): {('a', '<')},
 ('d', 'e', 'n', 'u'): {('d', 'n')},
 ('d', 't'): {('d', 't')},
 ('d', 'e', 'i', 'o'): {('d', '<')},
 ('d', 'u'): {('>', 'u')},
 ('d', 'n', 'o', 't'): {('>', 'o')},
 ('n', 'u'): {('u', 'u')},
 ('e', 'u'): {('e', 'u'), ('u', 'e')},
 ('t', 'u'): {('t', 'u'), ('u', 't')},
 ('i', 'u'): {('i', 'u'), ('u', 'i')},
 ('d', 'n', 'o', 'u'): {('u', 'o')},
 ('d', 'n', 'u'): {('u', '<')},
 ('a', 'e', 'n'): {('e', 'a')},
 ('e', 'n'): {('e', 'e')},
 ('a', 'e', 't'): {('e', 't')},
 ('d', 'e', 'i', 'n'): {('e', 'i'), ('i', 'e')},
 ('e', 'o'): {('e', 'o'), ('o', 'e')},
 ('e', 'n', 't'): {('t', 'n')},
 ('d', 'e', 't'): {('t', 'd')},
 ('d

## 3-MTSLIA gives a second opinion

In [119]:
mtslia(data, 3)

{('e', 'n', 'o'): {('e', 'n', 'o'),
  ('e', 'o', 'n'),
  ('n', 'e', 'o'),
  ('n', 'n', '<'),
  ('n', 'n', 'n'),
  ('n', 'o', 'e'),
  ('o', 'e', 'n'),
  ('o', 'n', 'e')},
 ('a', 'e', 'n'): {('a', 'e', 'n'),
  ('a', 'n', 'e'),
  ('e', 'a', '<'),
  ('e', 'a', 'n'),
  ('e', 'e', 'a'),
  ('n', 'a', 'e'),
  ('n', 'e', 'a'),
  ('n', 'n', 'a')},
 ('d', 'e', 'n'): {('>', '>', 'e'),
  ('>', 'e', 'n'),
  ('d', 'n', 'e'),
  ('e', 'd', 'n'),
  ('e', 'e', 'd'),
  ('e', 'n', 'd'),
  ('n', 'd', '<'),
  ('n', 'd', 'e'),
  ('n', 'e', '<'),
  ('n', 'n', 'd')},
 ('n', 'u'): {('>', 'n', 'u'),
  ('n', 'n', 'u'),
  ('n', 'u', '<'),
  ('n', 'u', 'u'),
  ('u', 'n', '<'),
  ('u', 'u', '<'),
  ('u', 'u', 'n')},
 ('e', 'i', 'n'): {('e', 'i', 'e'),
  ('e', 'i', 'n'),
  ('e', 'n', 'n'),
  ('i', 'e', 'e'),
  ('i', 'e', 'n'),
  ('i', 'i', 'e'),
  ('i', 'n', 'n'),
  ('n', 'i', 'e'),
  ('n', 'n', 'e')},
 ('a', 'i', 'n', 't'): {('n', 'n', 't')},
 ('d', 'i', 'n'): {('>', 'n', 'i'),
  ('d', 'i', 'n'),
  ('i', 'd', 'n'),
 

# A brute-force MTSL inference algorithm

Intentionally discards overlapping tiers. Has a very unwieldy section responsible for that.

In [120]:
def project(datum, k, tier, edges):
    result = tuple()
    for segment in datum:
        if segment in tier:
            result += (segment,)
    return (edges[0],) * (k - 1) + result + (edges[1],) * (k - 1)

def brute_force(data, k=2, edges=('>', '<')):
    alpha = tuple(sorted(extract_alphabet(data)))
    grammar = {}
    overlap_check = collections.defaultdict(set)
    for mask in itertools.product((True, False), repeat=len(alpha)):
        tier = tuple(itertools.compress(alpha, mask))
        unattested = set(kg for kg in itertools.product(tier + edges, repeat=k) if valid_k_gram(kg, edges))
        for datum in data:
            projection = project(datum, k, tier, edges)
            for offset in range(len(projection) - k + 1):
                k_gram = projection[offset:(offset + k)]
                if k_gram in unattested:
                    unattested.remove(k_gram)
        if unattested:
            # <unwieldy-section>
            skip = set()
            for kg in unattested:
                if overlap_check[kg]:
                    updated = True
                    while updated:
                        updated = False
                        for rival in set(overlap_check[kg]):
                            if not set(tier).issubset(rival) and not set(rival).issubset(tier):
                                if rival < tier:
                                    skip.add(kg)
                                else:
                                    grammar[rival].remove(kg)
                                    overlap_check[kg].remove(rival)
                                    updated = True
                                    break
                            else:
                                subset, superset = tier, rival
                                if set(superset).issubset(subset):
                                    subset, superset = superset, subset
                                if superset == tier:
                                    skip.add(kg)
                                else:
                                    grammar[superset].remove(kg)
                                    overlap_check[kg].remove(superset)
                                    updated = True
                                    break
                if kg not in skip:
                    overlap_check[kg].add(tier)
            # </unwieldy-section>
            grammar[tier] = unattested - skip
    return grammar

# Randomized shootout

Use alphabets that are prefixes of the following:

In [121]:
letters = 'aiuzsf'

We are interested in _k_ = 3:

In [122]:
k = 3

Define some testing machinery:

In [123]:
def scan(string, k, grammar, edges=('>', '<')):
    for tier in grammar:
        projection = project(string, k, tier, edges)
        for offset in range(len(projection) - k + 1):
            k_gram = projection[offset:(offset + k)]
            if k_gram in grammar[tier]:
                return False
    return True

def test(G1, G2, data, k, limit):
    alpha = extract_alphabet(data)
    for n in range(1, limit):
        for string in itertools.product(alpha, repeat=n):
            if scan(string, k, G1) != scan(string, k, G2):
                print(string, scan(string, k, G1), scan(string, k, G2))
                return False
    return True

Repeatedly feed our _k_-MTSLIA and the brute-force version random identical datasets:

In [124]:
from IPython.display import display, HTML

while True:
    # Alphabet size
    m = random.randint(2, len(letters) + 1)
    alpha = letters[:m]
    
    # Dataset size
    n = random.randint(1, 30)
    data = set()
    while len(data) < n:
        L = random.randint(1, 10)
        data.add(''.join(random.choice(alpha) for pos in range(L)))
    data = tuple(data)
    
    # Go!
    G1 = mtslia(data, k)
    G2 = brute_force(data, k)
    
    display(HTML('<h2>Dataset</h2>'))
    print('\n'.join(data))
    print()  

    # Print MTSLIA's constraints
    display(HTML('<h2><em>k</em>-MTSLIA</h2>'))
    for tier in sorted(G1):
        if not tier or not G1[tier]:
            continue
        display(HTML('<p style="color: teal; margin-bottom: 0;"><strong>Tier:</strong> %s</p>' % ', '.join(tier)))
        constraints = []
        for constraint in sorted(G1[tier]):
            constraints += ['*' + ''.join(constraint)]
        display(HTML('<p style="color: teal; margin-bottom: 10px;">%s</p>' % ', '.join(constraints)))
    print()
    
    # Print brute-force constraints
    display(HTML('<h2>Brute force</h2>'))
    for tier in sorted(G2):
        if not tier or not G2[tier]:
            continue
        display(HTML('<p style="color: purple; margin-bottom: 0;"><strong>Tier:</strong> %s</p>' % ', '.join(tier)))
        constraints = []
        for constraint in sorted(G2[tier]):
            constraints += ['*' + ''.join(constraint)]
        display(HTML('<p style="color: purple; margin-bottom: 10px;">%s</p>' % ', '.join(constraints)))
    print()
    
    display(HTML('<hr><hr>'))
    
    # Fingers crossed!
    if not test(G1, G2, data, k, 8):
        print('The last dataset produced a mismatch.')
        break

uuzassu
aaiia
a
iususszss
ziiiaiz
ziaiii
uuiaisaizu
aa
asssssz
sz
zuaiaza



zuffuffu
uiassisuai
zasizf
afzaiasz
f
fafu
sf
afaii
sszaf
zsas
iazzz



aiauai
uuiaaa
ua
iiuuiiui
uiuuiiaui
auaiiauua
uaai
uu



iaaiaiaa
iia
iiiaaii
iai
aaaiaa
ia
ai
iiiiiaaa
iaaia
aiiaaiiaii
iiaaa
iaaiiaia
aa
aiaaaa
aaaaiaai
iiiiiaia
iiiai
i
aiaiaa
aiaiaii



iauu
iaaa



sa
uzsaszuuia
fiaziza
asuszfizia
uuu
f
szifizuufi
saaffsi
saazu



izzzzaz
a
zazizaiiuz
uu
aiua
azzauaauiz
i
zizzuaaiua
uaizz
u
iiiuauuuu
aaiizuuzi
izuzaiaiiu
azz
zuiiiizi
zaiz
uzzzauzu
zuiuzauazu
azuzzui



z
usifsfszas
fu
ziufsfaaaz
szuissus
zsa
szfi
u
zzsazaaaza
f
fzszzaaa
siuiiu
s
uiuuafsi
iaus
fassuffss
fzzssuu
auusiu
zs
faz
uiuazfau
faussuzu
zizsssai
aaufu



KeyboardInterrupt: 